In [1]:
# from dotenv import load_dotenv
# load_dotenv()

In [2]:
import os
import requests
from typing import List, Dict, Any, TypedDict
from langchain_community.document_loaders import TextLoader, PyMuPDFLoader
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Weaviate
from langchain_openai import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.schema.runnable import RunnablePassthrough
from langgraph.graph import StateGraph, END

In [3]:
DO_INGEST = False
k = 20
n = 5

In [4]:
# text_splitter = CharacterTextSplitter(chunk_size=5000, chunk_overlap=100)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=6000, chunk_overlap=100)

In [5]:
import pymupdf

folder_path = "kb_ip"

text_content_dict = dict()

# Iterate through all files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith(".pdf"):  # Check if the file is a PDF
        pdf_path = os.path.join(folder_path, file_name)
        text_pdf = str()
        print(f"Processing: {file_name}")
        
        # Open the PDF file
        with pymupdf.open(pdf_path) as pdf_document:
            for page_num in range(len(pdf_document)):  # Loop through all pages
                page = pdf_document[page_num]
                text = page.get_text()  # Extract text from the page
                text_pdf += text
                # print(f"Page {page_num + 1}:\n{text}\n")

        # # Original file name
        # file_name = "example.txt"
        
        # # Split the file name into root and extension
        # file_root, file_ext = os.path.splitext(pdf_path)
        
        # # Replace the extension
        # new_file_name = f"{file_root}.txt"
        
        # with open(new_file_name, 'w', encoding='utf-8') as file:
        #     file.write(text_pdf)#.encode('utf-8'))
        
        loader = PyMuPDFLoader(pdf_path)
        documents = loader.load()
        chunks = text_splitter.split_documents(documents)
        
        text_content_dict[file_name] = chunks
        
        del loader, documents, chunks

Processing: 1757593739330.pdf
Processing: 2505.06416v1 SCALEMCP - DYNAMIC AND AUTO SYNCHRONIZING MODEL CONTEXT PROTOCOL TOOLS FOR LLM AGENTS.pdf
Processing: A Tutorial on LLM. Generative artificial intelligence… _ by Haifeng Li _ Medium.pdf
Processing: ACID Properties In DBMS Explained _ MongoDB _ MongoDB.pdf
Processing: Agentic_Design_Patterns.pdf
Processing: All you need to know about ‘Attention’ and ‘Transformers’ — In-depth Understanding — Part 1 _ by Arjun Sarkar _ Towards Data Science.pdf
Processing: All you need to know about ‘Attention’ and ‘Transformers’ — In-depth Understanding — Part 2 _ by Arjun Sarkar _ Towards Data Science.pdf
Processing: Applying word2vec to Recommenders and Advertising · Chris McCormick.pdf
Processing: Build your own Transformer from scratch using Pytorch _ by Arjun Sarkar _ Towards Data Science.pdf
Processing: Foundations of LLMs.pdf
Processing: linear algebra - How to intuitively understand eigenvalue and eigenvector_ - Mathematics Stack Exchange.pdf


In [6]:
# from pprint import pprint
# pprint(text_content_dict['1757593739330.pdf'])

In [7]:
documents = [value for sublist in text_content_dict.values() for value in sublist]
# documents

In [8]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

In [9]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [10]:
def custom_relevance_score_fn(cosine_dist):
    # return cosine_dist
    return 0
    # return 1-cosine_dist
    # return (cosine_dist + 1.0) / 2.0
    # return 1-((cosine_dist + 1.0) / 2.0)

In [11]:
# Create a ChromaDB vector store
vectorstore = Chroma(collection_name="my_rag_cosine", 
                     embedding_function=embeddings, 
                     persist_directory="chromadb_cosine",
                     relevance_score_fn=custom_relevance_score_fn,
                     collection_metadata={"hnsw:space": "cosine"}
                    )

In [12]:
if DO_INGEST:
    # vectorstore.add_texts([doc.page_content for doc in documents], metadatas=[doc.metadata for doc in documents])
    # vectorstore.add_documents(documents=documents)
    
    # Add documents in a loop
    print(f"Total Documents: {len(documents)}")
    for i in range(len(documents)):
        doc = documents[i]
        print(f"Ingesting Document: {i}")
        vectorstore.add_documents([doc])    
    print("Documents ingested to the vector store!")

else:
    print("No ingestion")

No ingestion


In [13]:
# # Similarity Search:

# results = vectorstore.similarity_search(
#     query="What is regularization?", k=k
# )

# for res in results:
#     print(f"Content: {res.page_content} \n\nMetadata: {res.metadata}\n\n\n\n")

In [14]:
# # Search by Vector:

# query_vector = embeddings.embed_query("What is regularization?")
# results = vectorstore.similarity_search_by_vector(embedding=query_vector, k=k)

# for res in results:
#     print(f"Content: {res.page_content} \n\nMetadata: {res.metadata}\n\n\n\n")

In [15]:
# Search by Score: [Without Re-ranking]

results = vectorstore.similarity_search_with_relevance_scores(query="What is regularization?", k=k)

for res in results:
    print(res)
    print(f"Content: {res[0].page_content} \n\nMetadata: {res[0].metadata}\n\n\n\n")

(Document(id='e33b18f1-b8db-45bd-b74b-4fe812ff7733', metadata={'page': 0, 'keywords': '', 'title': '', 'author': '', 'total_pages': 8, 'producer': 'Skia/PDF m120', 'creationDate': "D:20240127055328+00'00'", 'format': 'PDF 1.4', 'creationdate': '2024-01-27T05:53:28+00:00', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 Edg/120.0.0.0', 'modDate': "D:20240127055328+00'00'", 'source': 'kb_ip\\What is regularization_ _ IBM.pdf', 'moddate': '2024-01-27T05:53:28+00:00', 'subject': '', 'trapped': '', 'file_path': 'kb_ip\\What is regularization_ _ IBM.pdf'}, page_content='Explore watsonx.ai\nWhat is regularization?\nOverview'), 0)
Content: Explore watsonx.ai
What is regularization?
Overview 

Metadata: {'page': 0, 'keywords': '', 'title': '', 'author': '', 'total_pages': 8, 'producer': 'Skia/PDF m120', 'creationDate': "D:20240127055328+00'00'", 'format': 'PDF 1.4', 'creationdate': '2024-01-27T05:53:28+00:00', 'creator'

In [17]:
## Search by FlaskRank Re-Ranking

from flashrank import Ranker, RerankRequest
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank

base_retriever = vectorstore.as_retriever(search_kwargs={"k": 20})
print(f"Retrieved {k} documents for re-ranking.")

# Instantiate the Flashrank re-ranker
reranker = FlashrankRerank(model="ms-marco-MiniLM-L-12-v2", top_n=n)

# Wrap your base retriever with the re-ranker
compression_retriever = ContextualCompressionRetriever(
    base_compressor=reranker,
    base_retriever=base_retriever
)

# The query
query = "What is regularization?"

# Use the compression_retriever instead of the vector store directly
results = compression_retriever.invoke(query)

# Now, iterate through the re-ranked results
for res in results:
    # Print the document content and metadata, including the re-ranked score
    print(f"Content: {res.page_content} \n\nMetadata: {res.metadata}\n\n\n\n")

    # You can access the re-ranked score from the metadata
    reranked_score = res.metadata.get("relevance_score")
    if reranked_score:
        print(f"Re-ranked Score: {reranked_score}\n\n")




Retrieved 20 documents for re-ranking.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Content: Regularization. What, Why, When,
and How?
In this article, I want to take an in-depth look at regularization.
Akash Shastri · Follow
Published in Towards Data Science · 5 min read · Oct 24, 2020
132
What?
What is regularization?
Regularization is a method to constraint the model to fit our data accurately
and not overfit. It can also be thought of as penalizing unnecessary
complexity in our model. There are mainly 3 types of regularization
techniques deep learning practitioners use. They are:
1. L1 Regularization or Lasso regularization
2. L2 Regularization or Ridge regularization
3. Dropout
Sidebar: Other techniques can also have a regularizing effect on our model. You
can prevent overfitting by also having more data to constraint the search space of
our function. This can be done with techniques like data augmentation, that
create more data to train, hence reducing overfitting.
There are many other solutions to overfitting such as ensembling and stopping
early, that can help

In [18]:
# Search by GPT Re-ranking
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain.callbacks import get_openai_callback
from langchain.callbacks import OpenAICallbackHandler
callback_handler = OpenAICallbackHandler()

# Use a larger k for the initial retrieval
query = "What is regularization?"

candidate_documents = vectorstore.as_retriever(search_kwargs={"k": k}).invoke(query)
print(f"Retrieved {len(candidate_documents)} documents for re-ranking.")

def gpt_re_ranker(llm, query, documents, top_n=n):
    """
    Re-ranks documents using a GPT-based LLM.
    
    Args:
        llm: The LLM instance (e.g., ChatOpenAI).
        query: The user's query string.
        documents: A list of candidate documents (langchain.schema.Document).
        top_n: The number of top documents to return.

    Returns:
        A list of re-ranked documents.
    """
    
    # Prepare documents for the prompt
    docs_text = "\n\nDocument:".join(
        [f"Document {i+1}:\n{doc.page_content}" for i, doc in enumerate(documents)]
    )

    # Define the prompt for the LLM
    rerank_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are an expert at evaluating the relevance of document for the query. "
                "You have to rank documents based on relevance to a query i.e. how well and how completely the document answers the query. "
                "You will be given a list of documents and a query. "
                "Beginning of a new document is denoted by 'Document:' "
                "Rank the documents from most relevant to least relevant based on the query. "
                "Provide the output as a JSON array of objects, where each object has the document number and a relevance score (1-10). "
                "Example: [{{'doc_id': 1, 'score': 9}}, {{'doc_id': 3, 'score': 7}}]"
            ),
            ("human", "Query: {query}\n\nDocuments to rank:\n{docs_text}"),
        ]
    )

    # Create the chain for the re-ranker
    rerank_chain = rerank_prompt | llm | JsonOutputParser()

    # Invoke the chain to get the re-ranked results
    ranked_results = rerank_chain.invoke({"query": query, "docs_text": docs_text})
    
    # Sort the documents based on the re-ranked scores
    # Create a dictionary to map doc_id to its score
    doc_scores = {res['doc_id'] - 1: res['score'] for res in ranked_results}
    
    # Sort the original documents based on the new scores
    sorted_docs = sorted(documents, key=lambda doc: doc_scores.get(documents.index(doc), 0), reverse=True)
    
    # Return the top N documents
    return sorted_docs[:top_n]

# Instantiate your LLM
llm = ChatOpenAI(temperature=0,
                model="gpt-5-nano",
                max_tokens=None,
                timeout=None,
                max_retries=2,
                service_tier="flex",
                callbacks=[callback_handler])

# Use the custom re-ranker
reranked_docs = gpt_re_ranker(llm, query, candidate_documents, top_n=n)

### Step 3: Display the re-ranked documents
print("\n--- Re-ranked Documents ---")
for i, doc in enumerate(reranked_docs):
    score = doc.metadata.get("relevance_score", "N/A")
    print(f"Rank {i+1}:")
    print(f"Content: {doc.page_content} \nMetadata: {doc.metadata}\n\n")


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Retrieved 20 documents for re-ranking.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



--- Re-ranked Documents ---
Rank 1:
Content: Published: 16 November 2023
Contributors: Jacob Murel Ph.D., Eda Kavlakoglu
Regularization is a set of methods for reducing overfitting in machine
learning models. Typically, regularization trades a marginal decrease in
training accuracy for an increase in generalizability.
Regularization encompasses a range of techniques to correct for overfitting in machine
 As such, regularization is a method for increasing a model’s
learning models.
generalizability—that is, it’s ability to produce accurate predictions on new datasets.1
Regularization provides this increased generalizability at the sake of increased training
error. In other words, regularization methods typically lead to less accurate predictions
on training data but more accurate predictions on test data.
Regularization differs from optimization. Essentially, the former increases model
generalizability while the latter increases model training accuracy. Both are important
concepts in m

In [19]:
# Retrieve token usage and cost
print(f"Total Tokens Used: {callback_handler.total_tokens}")
print(f"Prompt Tokens: {callback_handler.prompt_tokens}")
print(f"Completion Tokens: {callback_handler.completion_tokens}")
print(f"Total Cost (USD): ${callback_handler.total_cost:.30f}")

Total Tokens Used: 9489
Prompt Tokens: 6510
Completion Tokens: 2979
Total Cost (USD): $0.000000000000000000000000000000
